In [ ]:
import json
import requests
import random
from tqdm import tqdm

In [ ]:
import json
import time

import pandas as pd
import requests
from tqdm import tqdm


def get_response_by_gpt4(prompt):
    headers = {
        'Authorization': 'sk-nYN1tlBh0oywCIy6480fDbC8D98d4581Ac6d640bFd8dD96b',
        'Content-Type': 'application/json',
    }
    json_data = {
        'model': 'gpt-3.5-turbo',
        'messages': [
            {
                'role': 'user',
                'content': prompt,
            },
        ],
        'stream': False,
    }
    
    try:
        response = requests.post('https://wdapi7.61798.cn/v1/chat/completions',
                                headers=headers,
                                json=json_data)
        result = response.json()
        return result["choices"][0]["message"]["content"]
    except BaseException:
        return ""

In [ ]:
PROMPT_TEMPLATE = """你是一个专业的中文对话文本句法依存分析语言专家，现在需要你来进行一些对话文本句法依存分析，以下是具体要求：
1. 输入的对话包含多轮，你需要对每一轮，每个词进行依存分析（每个词都存在依赖关系，不要遗漏！！！！），这些依赖关系可能跨多轮，输入的格式为：turn_编号:words_0_0,words_0_1,....words_x_x。
2. 输出是句法依存信息，格式为：(words_0_0,words_0_2,依赖关系)<>(words_0_1,words_0_3,依赖关系)<>...
3. 已知的依赖关系有（":"前边是标签名称,":"后面是标签的解释）：root:root, sasubj-obj:same subject, object, sasubj:same subject, dfsubj:different subject, subj:subject, subj-in: inner subject, obj:object
pred:predicate, att:attribute modifier, adv:adverbial modifier, cmp:complement modifier, coo:coordination, pobj:preposition object, iobj:indirect-object, de:de-construction, adjct:adjunct, app:appellation, exp:explanation, punc:punctuation, frag:fragment, repet:repetition
attr:attribution, bckg:background, cause:cause, comp:comparison, cond:condition, cont:contrast, elbr:elaboration, enbm:enablement, eval:evaluation, expl:explanation, joint:joint, manner:manner-means, rstm:restatement, temp:temporal, tp-chg:topic-change, prob-sol:problem-solution, qst-ans:question-answer, stm-rsp:statement-response, req-proc:requirement-process

以下是几个示例供你参考：
示例一：
{{example_01}}

示例二：
{{example_02}}

现在给你一个新的中文对话请你抽取其中的句法依存信息，再次强调输出格式为：(words_0_0,words_0_2,依赖关系)<>(words_0_1,words_0_3,依赖关系)<>...。
新对话：
{{dialog}}
"""

In [ ]:
with open("train_50.json", "r", encoding="utf8") as fi_train:
    train_data = json.load(fi_train)

In [ ]:
# 输入:turn_编号:words_0_0,words_0_1,....words_x_x。
# 输出:(words_0_0,words_0_2,依赖关系)<|reserved003|>(words_0_1,words_0_3,依赖关系)<|reserved003|>...
examples = []
for once_item in train_data:
    once_dialog_turns = once_item["dialog"]
    output_words_label = once_item["relationship"]
    input_text = ""
    turn_word_idx2words = {}
    for turn_idx, once_turn_words in enumerate(once_dialog_turns):
        input_text += f"turn_{turn_idx}:"
        words_text = []
        for word_idx, once_word in enumerate(
                once_turn_words["utterance"].split()):
            words_text.append(f"{once_word}_{turn_idx}_{word_idx}")
            turn_word_idx2words[f"{turn_idx}_{word_idx}"] = once_word
        input_text = input_text + ",".join(words_text) + "<n>"
    input_text = input_text.strip("<n>")
    output_text = []
    for once_rel in once_item["relationship"]:
        once_rel[0] = once_rel[0].replace("-", "_")
        once_rel[2] = once_rel[2].replace("-", "_")

        if once_rel[0] not in turn_word_idx2words or once_rel[
                2] not in turn_word_idx2words:
            continue
        once_word_0 = turn_word_idx2words[once_rel[0]]
        once_word_1 = turn_word_idx2words[once_rel[2]]
        output_text.append(
            f"({once_word_0}_{once_rel[0]},{once_word_1}_{once_rel[2]},{once_rel[1]})"
        )

    output_text = "<>".join(output_text)
    examples.append(f"INPUT:{input_text}\nOUTPUT:{output_text}")

In [ ]:
print(examples[0])

In [ ]:
# 加载新的对话数据集
with open("DuLeMon/both/dev.json", "r",
          encoding="utf-8") as fi_new_conversation:
    new_conversation_data = []
    for once_line in fi_new_conversation:
        conversation_ = json.loads(once_line)["conversation"]
        conversation_ = [_[4:].strip().split("\t")[0] for _ in conversation_]
        new_conversation_data.append(conversation_)

In [ ]:
labeled_datas = []
idx = 0
for once_conversation in tqdm(new_conversation_data[:500]):
    print("\n".join(once_conversation))
    dialog = ""
    for turn_idx, once_turn in enumerate(once_conversation):
        words_text = []
        for word_idx, once_word in enumerate(once_turn.split(" ")):
            words_text.append(f"{once_word}_{turn_idx}_{word_idx}")
        dialog += f"turn_{turn_idx}:" + ",".join(words_text) + "<n>"
    dialog = dialog.strip("<n>")
    dialog = f"INPUT:{dialog}\nOUTPUT:"
    random_samples = random.choices(examples, k=2)
    input_text = PROMPT_TEMPLATE.replace("{{example_01}}",
                                         random_samples[0]).replace(
                                             "{{example_02}}",
                                             random_samples[1])
    input_text = input_text.replace("{{dialog}}", dialog)

    output_text = get_response_by_gpt4(input_text)
    print(output_text)
    if len(output_text) == 0:
        continue
    labeled_datas.append([idx, once_conversation, dialog, output_text])
    idx += 1


In [ ]:
import pickle as pkl
with open("dev_labeled_datas.pkl", "wb") as fo_labeled_datas:
    pkl.dump(labeled_datas, fo_labeled_datas)
len(labeled_datas)